In [174]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)

def get_ret_year(mf_rets, st, ed, st1, ed1):
    mf_rets_year = mf_rets[(mf_rets['date']<=ed1) & (mf_rets['date']>=st)]
    date_ls = sorted(list(set(mf_rets_year['date'])))
    mf_rets_dicts = mf_rets_year.to_dict('records')
    rets_year_dict = {}
    for dict_i in mf_rets_dicts:
        crsp_fundno = dict_i['crsp_fundno']
        date = dict_i['date']
        ret = dict_i['mret']
        if crsp_fundno not in rets_year_dict:
            rets_year_dict[crsp_fundno] = {}
        rets_year_dict[crsp_fundno][date] = ret
    
    fundno_rets = {}
    NAN = float('nan')
    for fund_i in rets_year_dict:
        dict_i = rets_year_dict[fund_i]
        rest_ls = [NAN for i in range(len(date_ls))]
        for dete_i in dict_i:
            ind = date_ls.index(dete_i)
            rest_ls[ind] = dict_i[dete_i]
        fundno_rets[fund_i] = rest_ls
    rets_year = pd.DataFrame.from_dict(fundno_rets)
    rets_year = rets_year.dropna(axis=1, how='any')
    rets_year['date'] = date_ls
    rets_year1 = rets_year[(rets_year['date']<=ed) & (rets_year['date']>=st)]
    rets_year2 = rets_year[(rets_year['date']<=ed1) & (rets_year['date']>=st1)]
    rets_year1 = rets_year1.drop('date', axis=1)
    rets_year2 = rets_year2.drop('date', axis=1)

    return rets_year1, rets_year2


def get_qu_ls(quts, fund_retavg):
    ls = [[], [], [], [], [], [], [], [], [], []]
    for id_i in fund_retavg:
        avg = fund_retavg[id_i]
        for i in range(10):
            if (avg>=quts[i]) & (avg<quts[i+1]):
                ls[i].append(int(id_i))
    return ls
        
def get_group_ls(fundno_rets_year1):
    fund_retavg = fundno_rets_year1.mean().to_dict()
    ret_avgls = [fund_retavg[i] for i in fund_retavg]
    # from low to high:
    quts = list(np.percentile(ret_avgls, [(i+1)*10 for i in range(9)]))
    quts.insert(0, -8888)
    quts.append(8888)
    portfolios_fundids = get_qu_ls(quts, fund_retavg)
    return portfolios_fundids

def get_returns(ret_i):
    ret_array = ret_i.to_numpy()
    rest1 = ret_array[0, :]
    inval_indx0 = np.isnan(rest1)
    val_indx0 = 1- np.isnan(rest1)
    num_vals = np.sum(val_indx0)  
    ws = np.array([1/num_vals for i in range(rest1.shape[0])])
    ws[inval_indx0] = 0
    rets = []
    for i in range(12):
        rest1 = ret_array[i, :]
        rest11 = ret_array[i, :]
        rest11[np.isnan(rest11)] = 0
        ret_f1 = np.matrix(rest11)*np.matrix(ws).T
        ret_f1 = ret_f1[0, 0]
        rets.append(ret_f1)
        if i <= 10:
            rest2 = ret_array[i+1, :]
            indx_inval  = np.isnan(rest2)
            indx_val = 1 - indx_inval
            asets_vals = (np.array(rest11)+1)*np.array(ws)
            asets_vals = asets_vals + np.sum(asets_vals[indx_val])/np.sum(indx_val)
            asets_vals[indx_inval] = 0
            ws = asets_vals/np.sum(asets_vals)
            ws = ws.tolist()
    return rets


def get_pf_rets(pf_ids, fundno_rets_year2):
    ret_ls = []
    for pf_i in pf_ids:
        ret_i = fundno_rets_year2[pf_i]
        rets_pf_i = get_returns(ret_i)
        ret_ls.append(rets_pf_i)
    rets = np.array(ret_ls).T
    return rets


def get_all_rets(mf_rets):
    months  = []
    cols = ['p{}'.format(i+1) for i in range(10)]
    NANs = [float('nan') for i in range(12)]
    for i in tqdm(range(55)):
        st = int(str(1964+i)+'01')
        ed = int(str(1964+i)+'12')
        fundno_rets_year1 = get_ret_year(mf_rets, st, ed)
        pf_group_ls = get_group_ls(fundno_rets_year1)
        st1 = int(str(1964+i+1)+'01')
        ed1 = int(str(1964+i+1)+'12')
        fundno_rets_year2 = get_ret_year(mf_rets, st1, ed1)
        new_cols = set(fundno_rets_year1.columns) - set(fundno_rets_year2.columns)
        for col_i in new_cols:
            fundno_rets_year2[col_i] = NANs
        rets_i = get_pf_rets(pf_group_ls, fundno_rets_year2)
        if i==0:
            rets = rets_i
        else:
            rets = np.vstack((rets, rets_i))
        mon_i = [int(str(1964+i+1)+str(j+1).zfill(2)) for j in range(12)]
        months += mon_i
    df_pf_rets = pd.DataFrame(rets, columns=cols)
    df_pf_rets['date'] = months
    return df_pf_rets


def get_pf_rets1(pf_ids, fundno_rets_year2):
    ret_ls = []
    for pf_i in pf_ids:
        ret_i = fundno_rets_year2[pf_i]
        rets_pf_i = get_returns1(ret_i)
        ret_ls.append(rets_pf_i)
    rets = np.array(ret_ls).T
    return rets


def get_returns1(ret_i):
    ret_array = ret_i.to_numpy()
    rest1 = ret_array[0, :]
    num_vals = rest1.shape[0]
    ws = np.array([1/num_vals for i in range(num_vals)])
    rets = []
    for i in range(12):
        rest1 = ret_array[i, :]
        ret_f1 = np.matrix(rest1)*np.matrix(ws).T
        ret_f1 = ret_f1[0, 0]
        rets.append(ret_f1)
        if i <= 10:
            rest2 = ret_array[i+1, :]
            asets_vals = (np.array(rest1)/100+1)*np.array(ws)
            #print(np.sum(asets_vals))
            ws = asets_vals/np.sum(asets_vals)
            ws = ws.tolist()
    return rets

def get_all_rets1(mf_rets):
    months  = []
    cols = ['p{}'.format(i+1) for i in range(10)]
    NANs = [float('nan') for i in range(12)]
    for i in tqdm(range(54)):
        st = int(str(1964+i)+'01')
        ed = int(str(1964+i)+'12')
        st1 = int(str(1964+i+1)+'01')
        ed1 = int(str(1964+i+1)+'12')
        rets_year1, ret_year2 = get_ret_year(mf_rets, st, ed, st1, ed1)
        pf_group_ls = get_group_ls(rets_year1)
        rets_i = get_pf_rets1(pf_group_ls, ret_year2)
        if i==0:
            rets = rets_i
        else:
            rets = np.vstack((rets, rets_i))
        mon_i = [int(str(1964+i+1)+str(j+1).zfill(2)) for j in range(12)]
        months += mon_i
    df_pf_rets = pd.DataFrame(rets, columns=cols)
    df_pf_rets['date'] = months
    return df_pf_rets

# 1 process data

In [88]:
mf_rets = pd.read_csv('./original_data/MFrets_1962to2012.csv')
mf_rets = mf_rets.dropna(axis=0, how='any')
years = list(mf_rets['year'])
months = list(mf_rets['month'].astype(str))
dates = []
for i in range(len(years)):
    date_i = str(years[i]) + str(months[i]).zfill(2)
    dates.append(date_i)
mf_rets['date'] = dates
mf_rets['mret'] = mf_rets['mret']*100
mf_rets = mf_rets.drop(['year', 'month'], axis=1)
mf_rets.to_csv('./MF_rets.csv')

In [61]:
mom = pd.read_csv('./original_data/Mom.csv')
df3 = pd.read_csv('./original_data/FF3.csv')
df3 = df3[df3['Unnamed: 0']>=192701]
df3 = df3.reset_index()
df3['Mom'] = mom['Mom']
dates1 = sorted(list(set(dates)))
df3['date'] = df3['Unnamed: 0']
df3 = df3.drop(['Unnamed: 0', 'index'], axis=1)
df3.to_csv('FF4.csv')

# 2 regression:

$$
\begin{gathered}
r_{i t}=\alpha_{i T}+\beta_{i T} \mathrm{VWRF}_{t}+e_{i t} \quad t=1,2, \cdots, T \\
r_{i t}=\alpha_{i T}+b_{i T} \mathrm{RMRF}_{t}+s_{i T} \mathrm{SMB}_{t}+h_{i T} \mathrm{HML}_{t}+e_{i t} \quad t=1,2, \cdots, T \\
r_{i t}=\alpha_{i T}+b_{i T} \mathrm{RMRF}_{t}+s_{i T} \mathrm{SMB}_{t}+h_{i T} \mathrm{HML}_{t}+p_{i T} \mathrm{PR} 1 \mathrm{YR}_{t}+e_{i t} \\
t=1,2, \cdots, T
\end{gathered}
$$

## 2.1 variable meaning:

- $r_{i t}$ is the return on a portfolio in excess of the one-month T-bill return;
- VWRF is the excess return on the CRSP value-weighted portfolio of all NYSE;
- RMRF is the excess return on a value-weighted aggregate market proxy;
- SMB, HML, PR1YR are returns on valueweighted, zero-investment, factor-mimicking portfolios for size, book-to-market equity, and one-year momentum in stock returns.


## 2.2 portfolio  construction:

On January 1 of each year, I form ten equal-weighted portfolios of mutual funds, using reported returns.  I hold the portfolios for one year, then reform them. This yields a time series of monthly returns on each decile portfolio from 1963 to 1993. Funds that disappear during the course of the year are included in the equal-weighted average until they disappear, then the portfolio weights are readjusted appropriately.

Mutual funds are sorted on January 1 each year from 1963 to 1993 into decile portfolios based on their previous calendar year's return. The portfolios are equally weighted monthly so the weights are readjusted whenever a fund disappears. Funds with the highest past one-year return comprise decile 1 and funds with the lowest comprise decile 10. Deciles 1 and 10 are further subdivided into thirds on the same measure. VWRF is the excess return on the CRSP value-weight market proxy.

RMRF, SMB, and HML are Fama and French's (1993) market proxy and factor-mimicking portfolios for size and book-to-market equity. PR1YR is a factor-mimicking portfolio for one-year return momentum. Alpha is the intercept of the Model. The t-statistics are in parentheses.

In [145]:
ff4_df = pd.read_csv('./FF4.csv')
ff4_df = ff4_df.drop('Unnamed: 0', axis=1)
mf_rets = pd.read_csv('./MF_rets.csv')
mf_rets = mf_rets.drop('Unnamed: 0', axis=1)

In [175]:
df_pf_rets = get_all_rets1(mf_rets)

  9%|████                                        | 5/54 [00:00<00:02, 22.71it/s]

1.0572772857142856
1.0169557447304336
0.9804102313136669
1.0347187877905417
0.982118084422563
0.9133152123155723
1.015622878041819
1.0381450302024757
1.040288291895822
1.031337672073021
1.0334724156979385
1.0490119999999998
1.016296678142862
0.9803392397115307
1.0379444454325149
0.9731459921636386
0.9182002473498833
1.0281692252080763
1.048070603279305
1.0707825430200386
1.060703249793577
1.0739823696988036
1.0529188333333332
1.0161963083879686
0.9898128806387824
1.03706211727794
0.9727632261420986
0.907320606421923
1.0415020663224581
1.0532317763989967
1.0618026185129905
1.0483904830879651
1.0582569270457425
1.0545535
1.0249182779670891
0.9933729413892379
1.041476164373935
0.9982537988952945
0.9180985188406305
1.0393876654825753
1.0583393784164803
1.0402031267073457
1.0575663408169025
1.048045010555646
1.0621046666666665
1.0257452605104205
0.9936883368652921
1.0360341145589844
0.996791837166368
0.9243440334790991
1.0318831407181086
1.0519171916422398
1.040039402818569
1.04736552073177

 15%|██████▌                                     | 8/54 [00:00<00:02, 20.37it/s]

0.9212869285714285
1.0241359217558017
0.9678507967337512
0.8300186969588506
0.9101308921119263
0.9544098570809763
1.041053791559252
1.0314117991340384
1.0662395496400734
0.9726649054785764
1.0212637922700118
0.9129299999999999
1.0240312684900437
0.9551777462346085
0.821685136488182
0.9014238601696473
0.9436680954359056
1.03953252786818
1.0442613891459405
1.079013745542863
0.9569790962957077
1.0252474687988624
0.9277153846153846
1.052761747920284
0.9636574504260689
0.8410891235033588
0.9073237472277504
0.9604384209944
1.0487400554769113
1.0405568085966475
1.066134930912731
0.9755851569538108
1.0268742263339432
0.9194410000000001
1.048178757515959
0.9731572112720029
0.8510326014029956
0.913892469230579
0.9512634971129404
1.0595781216991997
1.0326298936871086
1.060201750389386
0.9674669824975666
1.0323326196485962
0.9284252307692308
1.0472655840228382
0.9787465241829985
0.8691064151106177
0.9199115222910688
0.9651426009467141
1.0448493083555954
1.0305981497507577
1.05765318145977
0.977196

 20%|████████▊                                  | 11/54 [00:00<00:02, 18.21it/s]

...

0.9497616688481676
1.030129871363238
1.070672347548149
1.060982029965892
0.9855316690444315
1.0330208807335377
1.0332934101761277
0.9833814826615355
1.015978934981622
0.9704694466361217
1.0350576108553735
0.9407818717277487
1.0040966542039063
1.066862684063278
1.0127511234256465
1.011824269363026
0.9975649601420422
1.038700958917281
1.007933460089212
1.0025267583693869
0.9740910228729123
1.065916279910049
0.9418167329842932
1.0009558702378492
1.0640839417532049
1.01169951836344
1.0138877038519243
0.997624317417171
1.0375137394303096
1.0069293419000598
1.0014362480050987
0.9763929872992396
1.0606223171784586
0.9435048468586387
0.9989595120428254
1.065103590191188
1.0106508654552244
1.0128151156395853
0.9963701243267278
1.0366395545812845
1.006001713269346
1.0010113945523964
0.9775844153690454
1.0511003119116393
0.947848802356021
0.9966759656829269
1.061953146321152
1.009680239627113
1.0113309406119813
0.9990918369849611
1.0345715139825078
1.003992857561194
1.0020203553039795
0.979274135

 98%|██████████████████████████████████████████▏| 53/54 [00:20<00:01,  1.03s/it]

1.0266583816120907
1.023099400157585
1.0073863066714803
1.0171408493548384
1.0152260366172556
1.0047523251261878
1.016848227553282
1.0104147715144305
1.007551278463997
1.0185527275259991
1.0146232741867576
1.0277891979823455
1.0277635621006744
1.0080387559784956
1.0170441267555912
1.0176907277814329
1.0023558658353855
1.0195154933585733
1.0073532656509765
1.0131874657782232
1.0214234996063192
1.0203523168716722
1.0190821815889028
1.0260644381078816
1.0030342828935384
1.012928214059459
1.012298184730882
1.004860289898018
1.0169514197754053
1.003732203044056
1.0131125216522094
1.0153451987908035
1.0192760300746304
1.021554522068096
1.0272616019979237
1.0051484564632283
1.0133959715094378
1.0137477599252143
1.005699303327595
1.0193809051445766
1.0035161371443182
1.0175040046342059
1.0184022689620689
1.0211922089153627
1.0209165636822193
1.0278525555843556
1.0045601656382113
1.012936696620764
1.0132477999137914
1.006789441034742
1.0198423917815245
1.0033813866706551
1.0199256462072124
1.01

100%|███████████████████████████████████████████| 54/54 [00:21<00:00,  2.55it/s]

0.9983371320049812
0.9693632016095703
1.0068853141750047
1.009945559835288
1.014617081676441
1.003048265072663
1.0025315728629782
1.0062184942600538
0.9926653867694062
0.9653140913947669
0.9980643589397153
1.0203759937655859
0.9601631308649285
1.0010382715759296
1.0043869069400544
1.021369608912098
1.0031191893932787
1.0154158736624503
1.013389080795498
0.990594418496882
0.9390383371744763
1.0072514301947675
1.0287126059850373
0.9592121071489871
0.9968517977602656
1.003598956425023
1.0211243460490484
1.0039460699386604
1.0218441494334973
1.0179480716586153
0.9913443455477557
0.9308173018600792
1.013173392280987
1.0331822577833125
0.9588712725156806
0.994119325727921
1.00335680440412
1.0196865071087644
1.0030171041793552
1.0243817103266464
1.0191154848167012
0.9928611616186132
0.9295515607749716
1.0164809135290165
1.040123561097257
0.9588103195505944
0.9892394283562376
1.0027379413294875
1.016389179963542
1.0004966231379213
1.0282888656067029
1.0193741989615759
0.9971856219377211
0.9301

In [176]:
df_pf_rets = df_pf_rets[df_pf_rets['date']<201901]
df_pf_rets['ls'] = df_pf_rets['p10'] - df_pf_rets['p1']
ff4_df1 = ff4_df[(ff4_df['date']>=196501) & (ff4_df['date']<=201812)]
ff4_df1 = ff4_df1.drop('date',axis=1)
ff4_df1 = ff4_df1.reset_index()

In [177]:
df_pf_rets[ff4_df1.columns[1:]] = ff4_df1[ff4_df1.columns[1:]]
df_pf_rets1 = df_pf_rets.copy()
cls = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'ls']
for cls_i in cls:
    df_pf_rets1[cls_i] = df_pf_rets1[cls_i] - df_pf_rets1['RF']
df_pf_rets1['MK'] = df_pf_rets1['Mkt-RF']
df_pf_rets2 = df_pf_rets1[df_pf_rets1['date']<=199312]
df_pf_rets3 = df_pf_rets1[(df_pf_rets1['date']>199312) & (df_pf_rets1['date']<=201812)]

In [178]:
df_pf_rets1.head(10)

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,date,ls,Mkt-RF,SMB,HML,RF,Mom,MK
0,5.447729,4.621200,5.011883,5.175350,5.930467,3.764986,3.623617,3.418033,4.770550,4.874929,196501,-0.852800,3.54,2.70,0.12,0.28,-1.44,3.54
1,1.395574,1.329668,1.319631,2.191828,2.274526,2.342554,1.562185,3.163301,1.998641,1.906774,196502,0.211199,0.44,3.53,0.11,0.30,0.32,0.44
2,-2.318977,-2.326076,-1.378712,-1.022706,-0.991166,-1.793139,-1.063565,-0.748955,-1.216096,0.066092,196503,2.025069,-1.34,1.88,1.03,0.36,0.15,-1.34
3,3.161879,3.484445,3.396212,3.837616,3.293411,3.245293,2.592714,3.294492,3.452637,3.455273,196504,-0.016606,3.11,1.15,0.66,0.31,2.63,3.11
4,-2.098192,-2.995401,-3.033677,-0.484620,-0.630816,-0.618600,-1.015758,-1.203141,-1.142813,-0.957528,196505,0.830664,-0.77,0.13,-1.61,0.31,0.60,-0.77
5,-9.018479,-8.529975,-9.617939,-8.540148,-7.915597,-7.651266,-5.557629,-8.576563,-7.049792,-7.486018,196506,1.182461,-5.51,-4.35,0.59,0.35,-3.13,-5.51
6,1.252288,2.506923,3.840207,3.628767,2.878314,2.247338,2.012421,2.885978,2.185071,0.891250,196507,-0.671038,1.43,0.87,2.14,0.31,4.12,1.43
7,3.484503,4.477060,4.993178,5.503938,4.861719,4.407778,2.718046,4.932389,4.258153,5.072875,196508,1.258372,2.73,2.83,-1.02,0.33,2.58,2.73
8,3.718829,6.768254,5.870262,3.710313,3.693940,3.307147,2.080733,3.560147,3.302717,3.543577,196509,-0.485253,2.86,0.64,-0.12,0.31,3.30,2.86
9,2.823767,5.760325,4.529048,5.446634,4.426552,4.040545,2.325615,5.939576,3.277796,3.208324,196510,0.074556,2.60,2.53,1.57,0.31,3.46,2.60


In [179]:
df_pf_rets1.to_csv('pf_rets1.csv')
df_pf_rets2.to_csv('pf_rets2.csv')
df_pf_rets3.to_csv('pf_rets3.csv')

In [180]:
cls = ['p10', 'p9', 'p8', 'p7', 'p6', 'p5', 'p4', 'p3', 'p2', 'p1', 'ls']

In [181]:
i = 0
for i in range(11):
    pi = cls[i]
    mean = round(df_pf_rets2[pi].mean(), 2)
    std = round(df_pf_rets2[pi].std(), 2)

    Y = df_pf_rets2[pi]
    X = df_pf_rets2['MK']
    X = sm.add_constant(X)
    mod_ff = sm.OLS(Y, X)
    ols_ff = mod_ff.fit()

    tvs = round(ols_ff.tvalues, 2)
    t1 = round(tvs[0], 2)
    t2 = round(tvs[1], 2)
    coefs = ols_ff.params
    coef1 = round(coefs[0], 2)
    coef2 = round(coefs[1], 2)

    AdjR2 = round(ols_ff.rsquared_adj, 2)

    X1 = df_pf_rets2[['MK', 'SMB', 'HML', 'Mom']]
    X1 = sm.add_constant(X1)
    mod_ff1 = sm.OLS(Y, X1)
    ols_ff1 = mod_ff1.fit()
    tvs1 = ols_ff1.tvalues
    t3 = round(tvs1[0], 2)
    t4 = round(tvs1[1], 2)
    t5 = round(tvs1[2], 2)
    t6 = round(tvs1[3], 2)
    t7 = round(tvs1[4], 2)

    coefs1 = ols_ff1.params
    coef11 = round(coefs1[0], 2)
    coef12 = round(coefs1[1], 2)
    coef13 = round(coefs1[2], 2)
    coef14 = round(coefs1[3], 2)
    coef15 = round(coefs1[4], 2)

    AdjR21 = round(ols_ff1.rsquared_adj, 2)
    line1 = '{}&{}&{}&{}&{}&{}&{}&{}&{}&{}&{}&{}\\\\'.format(i+1, mean, std, coef1, coef2, AdjR2, coef11, coef12, coef13, coef14, coef15, AdjR21)
    line2 = '&&&({})&({})&&({})&({})&({})&({})&({})&\\\\'.format(t1, t2, t3, t4, t5, t6, t7)
    print(line1)
    print(line2)

1&0.79&5.47&0.39&1.03&0.74&-0.06&0.87&0.65&-0.03&0.37&0.88\\
&&&(2.61)&(31.49)&&(-0.55)&(33.84)&(17.21)&(-0.68)&(12.48)&\\
2&0.66&5.2&0.24&1.06&0.86&0.04&0.92&0.44&-0.15&0.21&0.93\\
&&&(2.35)&(46.72)&&(0.53)&(50.77)&(16.4)&(-5.05)&(10.13)&\\
3&0.52&5.11&0.11&1.05&0.89&-0.07&0.93&0.37&-0.15&0.21&0.95\\
&&&(1.23)&(52.28)&&(-1.02)&(58.0)&(15.47)&(-5.81)&(11.23)&\\
4&0.4&4.99&-0.0&1.04&0.91&-0.16&0.93&0.35&-0.14&0.18&0.96\\
&&&(-0.03)&(58.45)&&(-2.71)&(67.4)&(17.06)&(-6.16)&(10.96)&\\
5&0.5&4.84&0.1&1.03&0.94&-0.01&0.93&0.28&-0.13&0.13&0.97\\
&&&(1.53)&(71.21)&&(-0.19)&(83.53)&(16.85)&(-7.1)&(10.4)&\\
6&0.33&4.74&-0.06&1.0&0.93&-0.12&0.91&0.25&-0.13&0.08&0.96\\
&&&(-0.96)&(69.41)&&(-2.21)&(71.3)&(13.39)&(-6.28)&(5.4)&\\
7&0.33&4.85&-0.07&1.03&0.94&-0.11&0.95&0.25&-0.1&0.04&0.96\\
&&&(-1.1)&(75.37)&&(-2.04)&(76.76)&(13.94)&(-4.65)&(2.53)&\\
8&0.32&4.91&-0.08&1.03&0.93&-0.09&0.95&0.26&-0.13&0.02&0.95\\
&&&(-1.1)&(65.47)&&(-1.33)&(62.65)&(11.58)&(-5.05)&(0.96)&\\
9&0.19&4.91&-0.21&1.02&0.9&-0

In [182]:
i = 0
for i in range(11):
    pi = cls[i]
    mean = round(df_pf_rets3[pi].mean(), 2)
    std = round(df_pf_rets3[pi].std(), 2)

    Y = df_pf_rets3[pi]
    X = df_pf_rets3['MK']
    X = sm.add_constant(X)
    mod_ff = sm.OLS(Y, X)
    ols_ff = mod_ff.fit()

    tvs = round(ols_ff.tvalues, 2)
    t1 = round(tvs[0], 2)
    t2 = round(tvs[1], 2)
    coefs = ols_ff.params
    coef1 = round(coefs[0], 2)
    coef2 = round(coefs[1], 2)

    AdjR2 = round(ols_ff.rsquared_adj, 2)

    X1 = df_pf_rets3[['MK', 'SMB', 'HML', 'Mom']]
    X1 = sm.add_constant(X1)
    mod_ff1 = sm.OLS(Y, X1)
    ols_ff1 = mod_ff1.fit()
    tvs1 = ols_ff1.tvalues
    t3 = round(tvs1[0], 2)
    t4 = round(tvs1[1], 2)
    t5 = round(tvs1[2], 2)
    t6 = round(tvs1[3], 2)
    t7 = round(tvs1[4], 2)

    coefs1 = ols_ff1.params
    coef11 = round(coefs1[0], 2)
    coef12 = round(coefs1[1], 2)
    coef13 = round(coefs1[2], 2)
    coef14 = round(coefs1[3], 2)
    coef15 = round(coefs1[4], 2)

    AdjR21 = round(ols_ff1.rsquared_adj, 2)
    line1 = '{}&{}&{}&{}&{}&{}&{}&{}&{}&{}&{}&{}\\\\'.format(i+1, mean, std, coef1, coef2, AdjR2, coef11, coef12, coef13, coef14, coef15, AdjR21)
    line2 = '&&&({})&({})&&({})&({})&({})&({})&({})&\\\\'.format(t1, t2, t3, t4, t5, t6, t7)
    print(line1)
    print(line2)


1&0.64&5.3&-0.0&1.02&0.68&-0.16&1.02&0.49&-0.12&0.3&0.89\\
&&&(-0.0)&(25.36)&&(-1.55)&(39.73)&(14.84)&(-3.24)&(13.73)&\\
2&0.61&4.68&-0.02&1.02&0.87&-0.12&1.01&0.33&0.04&0.15&0.95\\
&&&(-0.21)&(44.31)&&(-1.93)&(64.61)&(16.54)&(1.9)&(11.18)&\\
3&0.59&4.43&-0.04&1.0&0.94&-0.1&1.0&0.2&0.07&0.08&0.97\\
&&&(-0.64)&(71.27)&&(-2.41)&(93.14)&(14.39)&(4.75)&(8.76)&\\
4&0.53&4.29&-0.09&0.98&0.97&-0.13&0.98&0.13&0.08&0.05&0.98\\
&&&(-1.92)&(93.85)&&(-3.66)&(109.96)&(11.65)&(5.95)&(6.16)&\\
5&0.51&4.12&-0.08&0.95&0.98&-0.1&0.94&0.07&0.07&0.0&0.98\\
&&&(-2.3)&(112.34)&&(-2.94)&(112.68)&(6.64)&(5.67)&(0.24)&\\
6&0.46&4.14&-0.14&0.95&0.98&-0.12&0.94&0.04&0.03&-0.03&0.98\\
&&&(-3.62)&(109.93)&&(-3.43)&(105.83)&(3.89)&(2.75)&(-4.29)&\\
7&0.44&4.16&-0.16&0.95&0.96&-0.13&0.93&0.07&0.06&-0.06&0.97\\
&&&(-3.19)&(84.01)&&(-2.99)&(83.74)&(4.9)&(3.67)&(-5.86)&\\
8&0.43&4.16&-0.15&0.93&0.93&-0.13&0.91&0.06&0.08&-0.07&0.94\\
&&&(-2.42)&(64.59)&&(-2.16)&(63.33)&(3.01)&(3.81)&(-5.52)&\\
9&0.42&4.19&-0.15&0.92&0.8

In [183]:
i = 0
for i in range(11):
    pi = cls[i]
    mean = round(df_pf_rets1[pi].mean(), 2)
    std = round(df_pf_rets1[pi].std(), 2)

    Y = df_pf_rets1[pi]
    X = df_pf_rets1['MK']
    X = sm.add_constant(X)
    mod_ff = sm.OLS(Y, X)
    ols_ff = mod_ff.fit()

    tvs = round(ols_ff.tvalues, 2)
    t1 = round(tvs[0], 2)
    t2 = round(tvs[1], 2)
    coefs = ols_ff.params
    coef1 = round(coefs[0], 2)
    coef2 = round(coefs[1], 2)

    AdjR2 = round(ols_ff.rsquared_adj, 2)

    X1 = df_pf_rets1[['MK', 'SMB', 'HML', 'Mom']]
    X1 = sm.add_constant(X1)
    mod_ff1 = sm.OLS(Y, X1)
    ols_ff1 = mod_ff1.fit()
    tvs1 = ols_ff1.tvalues
    t3 = round(tvs1[0], 2)
    t4 = round(tvs1[1], 2)
    t5 = round(tvs1[2], 2)
    t6 = round(tvs1[3], 2)
    t7 = round(tvs1[4], 2)

    coefs1 = ols_ff1.params
    coef11 = round(coefs1[0], 2)
    coef12 = round(coefs1[1], 2)
    coef13 = round(coefs1[2], 2)
    coef14 = round(coefs1[3], 2)
    coef15 = round(coefs1[4], 2)

    AdjR21 = round(ols_ff1.rsquared_adj, 2)
    line1 = '{}&{}&{}&{}&{}&{}&{}&{}&{}&{}&{}&{}\\\\'.format(i+1, mean, std, coef1, coef2, AdjR2, coef11, coef12, coef13, coef14, coef15, AdjR21)
    line2 = '&&&({})&({})&&({})&({})&({})&({})&({})&\\\\'.format(t1, t2, t3, t4, t5, t6, t7)
    print(line1)
    print(line2)


1&0.72&5.39&0.21&1.03&0.71&-0.06&0.94&0.56&-0.06&0.32&0.88\\
&&&(1.84)&(40.17)&&(-0.8)&(52.09)&(22.4)&(-2.08)&(17.52)&\\
2&0.64&4.96&0.12&1.04&0.86&-0.03&0.98&0.36&-0.03&0.17&0.93\\
&&&(1.64)&(64.09)&&(-0.57)&(79.77)&(21.25)&(-1.78)&(13.48)&\\
3&0.55&4.81&0.04&1.03&0.91&-0.07&0.99&0.25&-0.02&0.12&0.95\\
&&&(0.68)&(80.37)&&(-1.55)&(91.6)&(16.59)&(-0.93)&(11.17)&\\
4&0.46&4.68&-0.05&1.02&0.93&-0.13&0.98&0.21&-0.01&0.09&0.95\\
&&&(-0.94)&(92.78)&&(-3.18)&(101.41)&(15.34)&(-0.38)&(8.96)&\\
5&0.5&4.52&0.01&0.99&0.95&-0.04&0.97&0.14&-0.01&0.05&0.96\\
&&&(0.25)&(110.46)&&(-1.07)&(111.88)&(11.99)&(-0.44)&(5.53)&\\
6&0.39&4.47&-0.1&0.98&0.95&-0.11&0.95&0.12&-0.03&0.0&0.96\\
&&&(-2.53)&(109.96)&&(-2.81)&(105.21)&(9.63)&(-1.81)&(0.44)&\\
7&0.38&4.54&-0.12&0.99&0.95&-0.12&0.96&0.14&0.0&-0.02&0.96\\
&&&(-2.8)&(107.89)&&(-2.95)&(104.43)&(10.89)&(0.08)&(-2.43)&\\
8&0.37&4.57&-0.12&0.99&0.93&-0.11&0.96&0.13&-0.0&-0.04&0.93\\
&&&(-2.44)&(89.87)&&(-2.23)&(84.36)&(8.33)&(-0.08)&(-3.29)&\\
9&0.3&4.59&-0.1